In [2]:
import sys 
import csv
sys.path.append("..")
from src.utils.constantes import DATA_CLEAN_PATH


1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.
Importante: tomar la información del último trimestre de cada año.

### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

12- CH05 date Fecha de nacimiento
(día, mes y año) --> ver como esta dado el dato

13- CH06 N (2) ¿Cuántos años cumplidos tiene?

16- CH09 N (1) ¿Sabe leer y escribir?
1 = Sí
2 = No
3 = Menor de 2 años

________________________________________________________
Columnas usadas: 2 - 1 - 9 - 13 - 16


In [ ]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_individual.csv"

def agrupar_por_anio(filas, col_anio=1):
    grupos = {}
    for fila in filas:
        anio = fila[col_anio]
        if anio not in grupos:
            grupos[anio] = []
        grupos[anio].append(fila)
    return grupos 




with archivo_clean_path.open("r",encoding="utf-8") as f:
    reader = csv.reader(f,delimiter=";")
    header = next(reader)
    new_list = list()
    for elem in reader:
        new_elem = elem[0].split(',')
        new_list.append(new_elem)
    dict_anios = agrupar_por_anio(new_list)
            
            
    

['TQRMNOSUWHLMKQCDEFNFF00852219', '2023', '4', '1', '2', '1', '41', 'N', '7', '370', '2', '2', '26/02/1971', '52', '2', '1', '1', '2', '0', '4', '1', '', '1', '', '1', '', '4', '3', '0', '4', '', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '0', '0', '0', '0', '', '0', '0', '370', '0', '0', '0', '0', '0', '', '0', '0', '370', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '12', '12', '12', '', '12', '12', '0', '12', '12', '12', '', '12', '12', '0', 'femenino', 'secundario completo', 'inactivo', '0']
